In [19]:
import torch
from torch import nn
from torch.optim import Adam
from model import Encoder, AttentionDecoder, EMBEDDING_SIZE, SOS_token
import random
from prepared import load_voc, batch2train_data

In [2]:
data, Vocabulary = load_voc()

## Подготовка данных, для использования в модели


In [3]:
inp, lenghts, target, mask, max_target_len = batch2train_data([random.choice(data) for _ in range(2)])

In [4]:
inp.size()

torch.Size([14, 2])

In [5]:
target.size()

torch.Size([12, 2])

In [42]:
device = "cuda:0"
batch_size = 64

In [43]:
embedding = nn.Embedding(2**16, EMBEDDING_SIZE).to(device)

encoder = Encoder(embedding).to(device)
decoder = AttentionDecoder(embedding).to(device)

encoder_optim = Adam(encoder.parameters(), lr=1e-4)
decoder_optim = Adam(decoder.parameters(), lr=5e-4)

In [44]:
def calculate_loss(inp, target, mask):

    loss = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = loss.masked_select(mask).mean()
    loss = loss.to(device)

    return loss

In [45]:
def train(epochs):

    for _ in range(epochs):

        encoder_optim.zero_grad()
        decoder_optim.zero_grad()

        inp, lenghts, target, mask, max_target_len = batch2train_data([random.choice(data) for _ in range(batch_size)])

        inp = inp.to(device)
        lenghts = lenghts.to(device)
        target = target.to(device)
        mask = mask.to(device)

        encoder_out, encoder_hidden = encoder(inp, lenghts)

        decoder_input = torch.ones(batch_size).long().to(device).unsqueeze(0)

        loss = 0

        for i in range(max_target_len):

            decoder_out, decoder_hidden = decoder(decoder_input, encoder_hidden[:2], encoder_out)

            decoder_input = target[i].view(1, -1)

            loss += calculate_loss(decoder_out, target[i], mask[i])

        loss.backward()

        encoder_optim.step()
        decoder_optim.step()

In [46]:
train(1000)

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 2.95 GiB total capacity; 1.79 GiB already allocated; 27.00 MiB free; 1.80 GiB reserved in total by PyTorch)